In [5]:
!pip install rank_bm25 nltk
!pip install pandas
!pip install numpy
!pip install html2text

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [17]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
import numpy as np
import pandas as pd
import html2text
from nltk.corpus import stopwords
import re

from IPython.display import HTML
#import nltk
#nltk.download('stopwords')
#nltk.download("punkt")

In [7]:
data = pd.read_csv('./data/messages.csv', header=None)
data.columns= ['subject','sender','to','date','empty','body']
data=data.drop(columns=['empty'])

In [8]:
data = data[data["to"]=="s.w.wintels@gmail.com"]
data = data.reset_index(drop=True)

In [9]:
from rank_bm25 import BM25Okapi

In [13]:
english_stopwords = list(set(stopwords.words('english')))

def tokenize(text):
    words = nltk.word_tokenize(text)
    return list(set([word for word in words if word.isalnum() 
                                             and not word in english_stopwords
                                             and not (word.isnumeric() and len(word) < 4)]))

In [14]:
h = html2text.HTML2Text()
h.ignore_links = False
#h.protect_links = True
#h.single_line_breaks = True
h.body_width=0

def t2h (string):
    return h.handle(string).replace("\n"," ")

def preprocess(string):
    string = t2h(string)
    return tokenize(string.lower())

In [15]:
def contains_words(needles, haystack):
    if "subject:" in haystack:
        return False
    for needle in needles:
        if needle in haystack:
            return True
    return False

In [18]:
corpus = [preprocess(body) for body in data["body"]]

In [19]:
bm25 = BM25Okapi(corpus)

In [22]:
def search(query,n=3):
    
    if len(query)<4:
        return pd.DataFrame()
    
    tokenized_query = query.lower().split(" ")
    doc_scores = bm25.get_scores(tokenized_query)
    
    top_x_results = np.argsort(doc_scores)[::-1][:n]
    top_x_scores = np.sort(doc_scores)[::-1][:n]
    
    results = []
    for j in range(len(top_x_results)):
        x = top_x_results[j]
        sentences = t2h(data.loc[x]['body']).split('---')
        good_sentences = [e for e in sentences if contains_words(tokenized_query, e.lower())]
        for sentence in good_sentences:
            sub_sentences = sentence.split('. ')
            good_sub_sentences = [e for e in sub_sentences if contains_words(tokenized_query, e.lower())]
            
            links=re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', sentence)
            for i in range(len(links)):
                    sentence = sentence.replace(links[i],f"<a target=_blank href='{links[i][:-1]}'>Link {i+1}</a>)")
            for sub_sentence in good_sub_sentences:
                sub_sentence = sub_sentence.strip().replace("|","")
                links=re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', sub_sentence)
                for i in range(len(links)):
                    sub_sentence = sub_sentence.replace(links[i],f"<a target=_blank href='{links[i][:-1]}'>Link {i+1}</a>)")
                    #links[i]=
                record = pd.Series([x,data["date"][x],top_x_scores[j],sub_sentence])#, sentence])
                results.append(record)
            
            
    results = pd.DataFrame(results)
    results.columns = ['id','date','score','sub_sentence']#,'paragraph']
    return results

In [23]:
from IPython.display import display
import ipywidgets as widgets
pd.options.display.max_colwidth = 500

def search_papers(SearchTerms: str):
    search_results = search(SearchTerms,20)
    if len(search_results) > 0:
        display(HTML(search_results.to_html(escape=False)))
        #display(search_results) 
    return search_results

searchbar = widgets.interactive(search_papers, SearchTerms='Walmart')
searchbar

interactive(children=(Text(value='Walmart', description='SearchTerms'), Output()), _dom_classes=('widget-inter…